In [2]:
class Permutation:
    def __init__(self):
        self.S = [
            0x8040201008040200,  # 64'd9241399655273594880
            0x00A04020100A0401,  # 64'd45400375717294337
            0xE7DAF6B48E7DAF38,  # 64'd16688268966856064344
            0x33DC56DC20D8DC16,  # 64'd3724962908607778966
            0x4C3DDBEE004DBA47   # 64'd5511705892194216375
        ]
        self.round = 6
        self.Cr = [
            0xF0, 0xE1, 0xD2, 0xC3, 0xB4, 0xA5, 0x96, 0x87, 0x78, 0x69, 0x5A, 0x4B
        ]
        self.const_table = [
            0x4, 0xB, 0x1F, 0x14, 0x1A, 0x15, 0x9, 0x2, 0x1B, 0x5, 0x8, 0x12,
            0x1D, 0x3, 0x6, 0x1C, 0x1E, 0x13, 0x7, 0xE, 0x0, 0xD, 0x11, 0x18,
            0x10, 0xC, 0x1, 0x19, 0x16, 0xA, 0xF, 0x17
        ]

    def substitution_layer(self, x0, x1, x2, x3, x4):
        x0_ps, x1_ps, x2_ps, x3_ps, x4_ps = 0, 0, 0, 0, 0
        for i in range(64):
            idx = ((x0 >> i) & 1) << 4 | ((x1 >> i) & 1) << 3 | ((x2 >> i) & 1) << 2 | ((x3 >> i) & 1) << 1 | ((x4 >> i) & 1)
            val = self.const_table[idx]
            x0_ps |= ((val >> 4) & 1) << i
            x1_ps |= ((val >> 3) & 1) << i
            x2_ps |= ((val >> 2) & 1) << i
            x3_ps |= ((val >> 1) & 1) << i
            x4_ps |= (val & 1) << i
        return x0_ps, x1_ps, x2_ps, x3_ps, x4_ps

    def linear_diffusion_layer(self, x):
        return x ^ (x >> 19) ^ (x >> 28) ^ (x << (64 - 19)) ^ (x << (64 - 28))

    def permute(self):
        x0, x1, x2, x3, x4 = self.S
        for counter in range(self.round):
            # Addition of Constants
            if self.round == 12:
                x2_pc = x2 ^ self.Cr[counter]
            elif self.round == 8:
                x2_pc = x2 ^ self.Cr[counter + 4]
            elif self.round == 6:
                x2_pc = x2 ^ self.Cr[counter + 6]
            else:
                x2_pc = x2

            # Substitution Layer
            x0_ps, x1_ps, x2_ps, x3_ps, x4_ps = self.substitution_layer(x0, x1, x2_pc, x3, x4)

            # Linear Diffusion Layer
            x0 = self.linear_diffusion_layer(x0_ps)
            x1 = self.linear_diffusion_layer(x1_ps)
            x2 = self.linear_diffusion_layer(x2_ps)
            x3 = self.linear_diffusion_layer(x3_ps)
            x4 = self.linear_diffusion_layer(x4_ps)

        return [x0, x1, x2, x3, x4]

if __name__ == "__main__":
    perm = Permutation()
    result = perm.permute()
    print("S_out:%h", result)


S_out:%h [357541662736510220499816455331394, 555694905469320471424635318401851, 455409873069201567304100553431326, 250348404506072348464358073259253, 126271191958379157338205757683777]
